# LangChain: Agents

Agents can receive a query by making use of a set of tools or resources at its disposal. These tools can include access to web search, Wikipedia, complex Mathematical libraries, LLMs, etc.

`Wikipedia-API` is easy to use Python wrapper for Wikipedias’ API. It supports extracting texts, sections, links, categories, translations, etc from Wikipedia. Documentation provides code snippets for the most common use cases.

In [1]:
%pip install -U wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=67fec3d89963318bcf3d9d8e4774eea3604ccc46a2b29f44c4558630aa38ab9b
  Stored in directory: /Users/ashique/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Built-in LangChain tools ✅

In [17]:
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import load_tools, initialize_agent
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
# from langchain.agents.agent_toolkits import create_python_agent

from utils import SaladChatOllama

In [11]:
llm = SaladChatOllama(temperature=0)

In [12]:
# Used to answer math questions and wikipedia articles related contents.
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [13]:
agent_predict = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/Users/ashique/Playground/LLMDeepDive/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  def emit_warning() -> None:


In [14]:
agent_predict("What is the 15% of 100?")

/Users/ashique/Playground/LLMDeepDive/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  def emit_warning() -> None:




> Entering new AgentExecutor chain...
Could not parse LLM output: Action: Calculator
Action_input: 100

$JSON_BLOB


Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Action: Wikipedia
Action_input: 15%

$JSON_BLOB


Observation: Invalid or incomplete response
Thought:Action: Calculator
Action_input: 100

Final Answer: 15

> Finished chain.


{'input': 'What is the 15% of 100?', 'output': '15'}

In [15]:
question = "Dr.APJ Abdul Kalam is an Indian Scientist and has served as President of India. What was his tenure as President of India ?"
result = agent_predict(question)



> Entering new AgentExecutor chain...
Could not parse LLM output: Action: Calculator
Action_input: 10

Answer: Dr. APJ Abdul Kalam's tenure as President of India was from 2002 to 2007, for a total of 5 years.
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Action: Wikipedia
Action_input: Dr. APJ Abdul Kalam

Answer: Dr. APJ Abdul Kalam served as the 11th President of India from May 2002 to May 2007, for a total of 5 years.
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Action: Calculator
Action_input: 10

Answer: Dr. APJ Abdul Kalam's tenure as President of India was from 2002 to 2007, for a total of 5 years.

Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Action: Wikipedia
Action_input: Dr. APJ Abdul Kalam

Answer: Dr. APJ Abdul Kalam served as the 11th President of India from May 2002 to May 2007, for a total of 5 years.


Observation: Invalid or incomplete response
Thought:Could not p

## Python Agent

In [18]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [19]:
customer_list = [["Naveen", "40"],
                 ["Balaji", "40"],
                 ["Aashique", "30"],
                 ["Adarsh", "20"],
                 ["Sai","15"],
                 ["Sukan","10"],
                 ["Aashique","40"],
                 ["Sai","19"]
                ]

In [22]:
agent.run(f"""Sort the below data, based on the names and then age \
print output: {customer_list}""")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Action: `sort([['Naveen', '40'], ['Balaji', '40'], ['Aashique', '30'], ['Adarsh', '20'], ['Sai', '15'], ['Sukan', '10']])``

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True

agent.run(f"""Sort the below data, based on the names and then age \
print output: {customer_list}""")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort the below data, based on the names and then age print output: [['Naveen', '40'], ['Balaji', '40'], ['Aashique', '30'], ['Adarsh', '20'], ['Sai', '15'], ['Sukan', '10'], ['Aashique', '40'], ['Sai', '19']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort the below data, based on the names and then age print output: [['Naveen', '40'], ['Balaji', '40'], ['Aashique', '30'], ['Adarsh', '20'], ['Sai', '15'], ['Sukan', '10'], ['Aashique', '40'], ['Sai', '19']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

"[['Aashique', '30'], ['Aashique', '40'], ['Adarsh', '20'], ['Balaji', '40'], ['Naveen', '40'], ['Sai', '15'], ['Sai', '19'], ['Sukan', '10']]"

## Create your Own Tool ✅

In [23]:
import random
from langchain.agents import tool

In [24]:
@tool
def get_inspiring_quote(text: str) -> str:
    """Returns a random inspirational quote from the below quotes."""
    quotes = [
        "Believe you can and you're halfway there. -Theodore Roosevelt",
        "The only limit to our realization of tomorrow will be our doubts of today. -Franklin D. Roosevelt",
        "Success is not final, failure is not fatal: It is the courage to continue that counts. -Winston Churchill",
        "The only way to do great work is to love what you do. -Steve Jobs",
        "In the middle of every difficulty lies opportunity. -Albert Einstein"
    ]

    return random.choice(quotes)

In [25]:
agent= initialize_agent(
    tools + [get_inspiring_quote],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

*To Note*:

The agent may occasionally arrive at an inaccurate conclusion (agents are continuously evolving and improving!).

If such a situation arises, please consider rerunning the query for a more accurate response.

In [26]:
try:
    result = agent("Give me a motivational quote for today")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Could not parse LLM output:  Action: get_inspiring_quote(text: str) -> str

Thought: Let's see what kind of inspirational quote I can find for you today! 😊


Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Action: get_inspiring_quote(text: str) -> str

Thought: Here's a beautiful and inspiring quote for you today: "Believe you can and you're halfway there." - Theodore Roosevelt 💫


Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Question: What is the best way to approach a challenging situation?

Action: Use get_inspiring_quote() to find an inspirational quote.

Thought: Let's see what kind of wisdom the universe has in store for us today... 🤔


Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Great, let's get started! Please provide me with a question or topic you would like to learn more about, and I will use the tools at my disposal to provide you wi